# <a id='toc1_'></a>[Run Whisper Medium Shqip](#toc0_)

Ky model është i trajnuar në gjuhën shqipe dhe mund të përdoret për të transkriptuar zërin në tekst. Është i bazuar në modelin [openai/whisper-medium](https://huggingface.co/openai/whisper-medium) i zhvilluar nga OpenAI. 

Hapat para se të përdorni këtë model janë:

1. Krijo një llogari në Hugging Face (nëse nuk keni një).
2. Krijo një token për autentifikim (API token) në [Settings -> Access Tokens](https://huggingface.co/settings/tokens)
3. Kopjo tokenin dhe ruaje në një skedar teksti.
4. Shko në [Kushtrim/whisper-medium-sq](https://huggingface.co/Kushtrim/whisper-medium-sq) dhe kërko autorizim për përdorim.

**Table of contents**<a id='toc0_'></a>    
- [Run Whisper Medium Shqip](#toc1_)    
  - [Install libraries](#toc1_1_)    
  - [Load libraries](#toc1_2_)    
  - [HuggingFace Login](#toc1_3_)    
  - [Load model](#toc1_4_)    
  - [Transcribe audio](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Install libraries](#toc0_)

In [ ]:
!pip install --upgrade --quiet transformers huggingface-hub datasets

## <a id='toc1_2_'></a>[Load libraries](#toc0_)

In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from huggingface_hub import notebook_login
import requests
import torch

## <a id='toc1_3_'></a>[HuggingFace Login](#toc0_)

In [ ]:
# vendos access token ketu:
notebook_login()

## <a id='toc1_4_'></a>[Load model](#toc0_)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "Kushtrim/whisper-medium-sq"
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, use_safetensors=True).to(device)
processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline("automatic-speech-recognition", model=model, tokenizer=processor.tokenizer, feature_extractor=processor.feature_extractor,
                max_new_tokens=128, chunk_length_s=15, batch_size=16, torch_dtype=torch_dtype, device=device)

## <a id='toc1_5_'></a>[Transcribe audio](#toc0_)

In [ ]:
# Download a sample

def download_file(url, local_filename):
    # Send a GET request to the URL
    response = requests.get(url, stream=True)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open a local file with write-binary mode
        with open(local_filename, 'wb') as f:
            # Write the content of the response in chunks
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"File downloaded successfully: {local_filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")


# Example usage
url = 'https://github.com/KushtrimVisoka/datasets/raw/main/deklarata-e-pavaresise-se-kosoves.mp3'
local_filename = 'deklarata-e-pavaresise-se-kosoves.mp3'
download_file(url, local_filename)


In [ ]:
results = pipe("deklarata-e-pavaresise-se-kosoves.mp3", return_timestamps=True, generate_kwargs={'task': 'transcribe', 'language': 'sq'})
print(results)